## Return value of wav dataset (format of tf.data for training)

In [1]:
import tensorflow as tf
wav_ds = tf.keras.utils.audio_dataset_from_directory('/media/jacek/E753-A120/wav_rec_16')
for features in wav_ds.take(3):
    print(features)
    print("\n\n")

2023-11-02 11:54:30.978327: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-02 11:54:30.980302: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 11:54:31.019465: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 11:54:31.020059: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 11:54:32.047260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Found 3 files belonging to 2 classes.


2023-11-02 11:54:32.976116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-02 11:54:32.976775: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


(<tf.Tensor: shape=(3, 5556600, 2), dtype=float32, numpy=
array([[[ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        ...,
        [ 0.0000000e+00,  3.0517578e-05],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 3.0517578e-05, -3.0517578e-05]],

       [[ 1.4648438e-03,  0.0000000e+00],
        [ 8.8806152e-03,  0.0000000e+00],
        [ 4.4250488e-03,  0.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00]],

       [[ 2.1972656e-03,  2.1667480e-03],
        [ 1.9836426e-03,  2.0751953e-03],
        [ 1.9531250e-03,  1.8005371e-03],
        ...,
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00]]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 0, 1], dtype=int32)>)




In [2]:
wav_ds

<_PaddedBatchDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

## Custom dataset from mp3 (only filepath)

### Config + import + functions

In [3]:

import librosa.display
import tensorflow as tf
from birdclassification.preprocessing.filtering import filter_recordings_30
from birdclassification.visualization.plots import plot_waveform
from sklearn.preprocessing import LabelEncoder

AUTOTUNE = tf.data.experimental.AUTOTUNE
N_CLASSES = 264  # Allows to reduce the number of classes to train (max = 264)
SAMPLE_RATE = 32000  # Audio sample rate
MAX_DURATION = 5  # Clip duration in seconds 
FFT_SIZE = 1024  # Fourier Transform size 
HOP_SIZE = 512  # Number of samples between each successive FFT window
N_MEL_BINS = 128
N_SPECTROGRAM_BINS = (FFT_SIZE // 2) + 1
F_MIN = 20  # Min frequency cutoff
F_MAX = SAMPLE_RATE / 2  # Max Frequency cutoff
BATCH_SIZE = 64  # Training Batch size
DATA_PATH = "/media/jacek/E753-A120/recordings_30/"

### Prepare dataframe

In [4]:
df = filter_recordings_30()
df['filepath'] = df.apply(lambda x: f"{DATA_PATH}{x['Latin name']}/{str(x['id'])}.mp3" , axis=1)
#df['Latin name'].value_counts()[:N_CLASSES]
le = LabelEncoder()
df['label'] = le.fit_transform(df['Latin name'])
df

/home/jacek/PycharmProjects/Bird-classification-model/birdclassification/preprocessing/filtering.py:43: DtypeWarning: Columns (10,39,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  recordings = pd.read_csv("../data/xeno_canto_recordings.csv")


,index,Unnamed: 0,id,gen,sp,ssp,group,en,rec,cnt,...,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Latin name,filepath,label
0,5,5,815762,Cygnus,olor,NaN,birds,Mute Swan,Frédéric Cazaban,France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cygnus olor,/media/jacek/E753-A120/recordings_30/Cygnus ol...,12
1,6,6,808540,Cygnus,olor,NaN,birds,Mute Swan,Juha Saari,Finland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cygnus olor,/media/jacek/E753-A120/recordings_30/Cygnus ol...,12
2,7,7,805048,Cygnus,olor,NaN,birds,Mute Swan,Magnus Hellström,Sweden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cygnus olor,/media/jacek/E753-A120/recordings_30/Cygnus ol...,12
3,8,8,804791,Cygnus,olor,NaN,birds,Mute Swan,Uku Paal,Estonia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cygnus olor,/media/jacek/E753-A120/recordings_30/Cygnus ol...,12
4,9,9,797609,Cygnus,olor,NaN,birds,Mute Swan,Jorge Leitão,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cygnus olor,/media/jacek/E753-A120/recordings_30/Cygnus ol...,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41205,296788,296788,108758,Columba,livia,NaN,birds,Rock Dove,Albert Lastukhin,Venezuela,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columba livia,/media/jacek/E753-A120/recordings_30/Columba l...,7
41206,296789,296789,58488,Columba,livia,NaN,birds,Rock Dove,Mike Nelson,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columba livia,/media/jacek/E753-A120/recordings_30/Columba l...,7
41207,296790,296790,6924,Columba,livia,domestica,birds,Rock Dove,Glauco Alves Pereira,Brazil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columba livia,/media/jacek/E753-A120/recordings_30/Columba l...,7
41208,296791,296791,387816,Columba,livia,NaN,birds,Rock Dove,johannes buhl,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columba livia,/media/jacek/E753-A120/recordings_30/Columba l...,7


### Split and create tf.data

In [5]:
def get_dataset(df, label_column, filepath_column):
    file_path_ds = tf.data.Dataset.from_tensor_slices(df[filepath_column].astype(bytes))
    return file_path_ds
    #label_ds = tf.data.Dataset.from_tensor_slices(df[label_column])
    #return tf.data.Dataset.zip((file_path_ds, label_ds))


from sklearn.model_selection import train_test_split
train_df, test_val_df = train_test_split(df, stratify=df['Latin name'], test_size=0.2)
val_df, test_df = train_test_split(test_val_df, stratify=test_val_df['Latin name'], test_size=0.5)

train_ds = get_dataset(train_df, 'label', 'filepath')
val_ds = get_dataset(val_df, 'label', 'filepath')
test_ds = get_dataset(test_df, 'label', 'filepath')

In [6]:
train_ds

for features in train_ds.take(2):
    print(features)

tf.Tensor(b'/media/jacek/E753-A120/recordings_30/Hirundo rustica/662037.mp3', shape=(), dtype=string)
tf.Tensor(b'/media/jacek/E753-A120/recordings_30/Turdus merula/683340.mp3', shape=(), dtype=string)


## How to load?


In [9]:
import librosa
import tensorflow as tf

filepath_ds = tf.data.Dataset.list_files('/media/jacek/E753-A120/recordings_30/*/*')
label_ds = tf.data.Dataset.from_tensor_slices(df['label'])
zip_ds = tf.data.Dataset.zip((filepath_ds, label_ds))

def decode_audio(path):
    path = path.numpy().decode("utf-8")
    audio, sr = librosa.load(path)
    return audio

zip_ds_2 = zip_ds.map(lambda path, label: (tf.py_function(func=decode_audio, inp=[path], Tout=tf.float32), label))

# for features in zip_ds_2.take(3):
#     print(features)
# 
# for features in wav_ds.take(3):
#     print(features)

zip_ds_2 = zip_ds_2.batch(64).prefetch(AUTOTUNE)
print(zip_ds_2)
print(wav_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
<_PaddedBatchDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
